## Understanding Algorand Addresses
#### 03.2 Winter School on Smart Contracts
##### Peter Gruber (peter.gruber@usi.ch)
2023-02-07 (started 2022-01-05)

* The nature and dimension of Algorand addresses
* Encodings: Hex, Base32, Base 64
* How to decode and understand Algorand addresses
* How the mnemonic passphrase works

In [18]:
import base64
import algosdk

## Algorand addresses
* Private key is the basis
* Length is 32 bytes = 256 bits
* How many different addresses are there? $2^{256}$

In [19]:
combinations = 2**256                     
print(combinations)                     # very hard to read number

115792089237316195423570985008687907853269984665640564039457584007913129639936


In [20]:
f"{combinations:1.3E}"

'1.158E+77'

Compare this to the number of atoms in the observable universe, which is estimated to be between $10^{70}$ and $10^{80}$

#### Cracking algorand
If you can guess 1 million keys per second, how long would it take to guess all of them?

In [21]:
guess = 1E6
print( f"{combinations/guess :1.3E} seconds" )
print( f"{combinations/guess/60/60/24 :1.3E} days" )
print( f"{combinations/guess/60/60/24/365 :1.3E} years" )

1.158E+71 seconds
1.340E+66 days
3.672E+63 years


**EXERCISE:** How long would it take if our computer were 1000 times faster? And if we could use all estimated 5 billion computers in the world?

## *Encodings
* A byte can take $2^8 = 256$ different values
    * Decimal representation requires 3 digits
* Encoding = find a way to represent one byte with fewer symbols
* Encodings make a compromise between efficiency and readability
* Below some general examples of decoding and encoding strings and numbers in Python

#### (1) Hex encoding
* Find a way to represent one byte with only 2 symbols
+ Using numbers `0`-`9` plus letters `a`-`f` $\rightarrow$ 16 symbols
+ One symbol encodes $16=2^4$ values $\rightarrow$ 4 bits
+ Two symbols encode $16 \cdot 16 = 2^8 = 256$ values  $\rightarrow$ 8 bits = 1 byte

In [26]:
numbers = [0, 1, 10, 15, 16, 17, 32]
print(numbers)
[hex(number) for number in numbers]

[0, 1, 10, 15, 16, 17, 32]


['0x0', '0x1', '0xa', '0xf', '0x10', '0x11', '0x20']

#### (2) Base 32 encoding
+ Using 32 symbols, *not* standardized
+ One symbol has $32 = 2^5$ possibilities $\rightarrow$ 5 bits
+ Example: numbers `0`-`9` and letters `A`-`V`  (RFC 4648)


In [27]:
numbers = [0, 1, 10, 16, 31, 32, 1023]
print(numbers)
[base64.b32hexencode(number.to_bytes(5, 'big')) for number in numbers]

[0, 1, 10, 16, 31, 32, 1023]


AttributeError: module 'base64' has no attribute 'b32hexencode'

#### (3) Base 64 encoding
+ Using 64 symbols, *not* standardized
+ One symbol has $64 = 2^6$ possibilities $\rightarrow$ 6 bits
* Example: letters `A`-`Z`, `a`-`z`, numbers `0`-`9` and special characters `+`, `/`

In [13]:
numbers = [0, 1, 25, 26, 51, 52, 61, 62, 63, 64]
print(numbers)
[base64.standard_b64encode(number.to_bytes(3, 'big')) for number in numbers]

[0, 1, 25, 26, 51, 52, 61, 62, 63, 64]


[b'AAAA',
 b'AAAB',
 b'AAAZ',
 b'AAAa',
 b'AAAz',
 b'AAA0',
 b'AAA9',
 b'AAA+',
 b'AAA/',
 b'AABA']

### Algorand address encoding

In [28]:
# Account info generation.
private_key, public_key = algosdk.account.generate_account()

##### Private key
Algorand stores private key (256 bits) with public key (256 bits) in base64 encoding

In [29]:
print( private_key )
print( len(private_key) )
print( (len(private_key)-2) * 6 )           # 6bits per symbol, two adresses encoded, last two symbols ("==") do not count

Jdij0P2Ng16wJ0OHNASNt/OADZLj4otn5XxplSWd6zTeSrrq2ayN9slDQUBAP4On+k1RiJQNqEncS8G7LhTrog==
88
516


##### Public key
Algorand stores public key (256 bits) plus a 4 byte checksum (32 bits) in base32 encoding

In [30]:
print( public_key )
print( len(public_key) )
len(public_key) * 5                        # 5bits per symbol, 256+32 = 288

3ZFLV2WZVSG7NSKDIFAEAP4DU75E2UMISQG2QSO4JPA3WLQU5ORIEBMKUM
58


290

### *Inside the address
* Keys and addresses are actually very long nubers
* Can be represented in several ways

In [31]:
# Standard base32 representation
print(public_key)
print('')

# The decoded address is only 32 Bytes long
decoded_addr = algosdk.encoding.decode_address(public_key)
print(len(decoded_addr))
print('')

# Decimal number
print(int.from_bytes(decoded_addr,byteorder='big'))
print('')

# All 32 bytes of the address
for my_byte in decoded_addr:
    print(f'{my_byte:d}', end=' ')
print('\n')

# All 256 bits of the address
for my_byte in decoded_addr:
    print(f'{my_byte:>08b}', end=' ')
print('\n')

# Hex repsresentation
print(decoded_addr.hex())

3ZFLV2WZVSG7NSKDIFAEAP4DU75E2UMISQG2QSO4JPA3WLQU5ORIEBMKUM

32

100545489124650513932789238587753846232098919517433246784652650140936455908258

222 74 186 234 217 172 141 246 201 67 65 64 64 63 131 167 250 77 81 136 148 13 168 73 220 75 193 187 46 20 235 162 

11011110 01001010 10111010 11101010 11011001 10101100 10001101 11110110 11001001 01000011 01000001 01000000 01000000 00111111 10000011 10100111 11111010 01001101 01010001 10001000 10010100 00001101 10101000 01001001 11011100 01001011 11000001 10111011 00101110 00010100 11101011 10100010 

de4abaead9ac8df6c9434140403f83a7fa4d5188940da849dc4bc1bb2e14eba2


## The Mnemonic key or passphrase
Consists of 25 words, each chose of a list of $2^{11} = 2048$ words. The encoding is simple, we chosse the *n-th* word in the **bip-0039 English word list**<br>
Each word encodes 11 bits, total is

In [32]:
25 * 11

275

24 words would be enough, but again there is 2-byte hash of the private key

In [33]:
25 * 11 - 16

259

#### All 2048 words from the BIP 39  english wordlist
* BIP stands for Bitcoin Improval Proposal.
* BIP 39 was proposed in 2013
* BIP 39 has its critics, because there are many variants of the standard, see https://electrum.readthedocs.io/en/latest/seedphrase.html#motivation

**Hint** Click into the blue bar to hide the output.

In [34]:
print(algosdk.wordlist.word_list_raw())

abandon
ability
able
about
above
absent
absorb
abstract
absurd
abuse
access
accident
account
accuse
achieve
acid
acoustic
acquire
across
act
action
actor
actress
actual
adapt
add
addict
address
adjust
admit
adult
advance
advice
aerobic
affair
afford
afraid
again
age
agent
agree
ahead
aim
air
airport
aisle
alarm
album
alcohol
alert
alien
all
alley
allow
almost
alone
alpha
already
also
alter
always
amateur
amazing
among
amount
amused
analyst
anchor
ancient
anger
angle
angry
animal
ankle
announce
annual
another
answer
antenna
antique
anxiety
any
apart
apology
appear
apple
approve
april
arch
arctic
area
arena
argue
arm
armed
armor
army
around
arrange
arrest
arrive
arrow
art
artefact
artist
artwork
ask
aspect
assault
asset
assist
assume
asthma
athlete
atom
attack
attend
attitude
attract
auction
audit
august
aunt
author
auto
autumn
average
avocado
avoid
awake
aware
away
awesome
awful
awkward
axis
baby
bachelor
bacon
badge
bag
balance
balcony
ball
bamboo
banana
banner
bar
barely
bargain
barre

### Only the first four letters matter
The function `mnemonic.to_private_key` works with full words or only the first four letters 

#### (a) using full words

In [35]:
passphrase = algosdk.mnemonic.from_private_key(private_key)
print(passphrase)
algosdk.mnemonic.to_private_key(passphrase)

again month tribe thank alpha bless diesel drip movie awkward shoot derive acid lock broken title vapor night salmon final enroll trap point ability pledge


'Jdij0P2Ng16wJ0OHNASNt/OADZLj4otn5XxplSWd6zTeSrrq2ayN9slDQUBAP4On+k1RiJQNqEncS8G7LhTrog=='

#### (b) using only the first four letters

In [36]:
# A quick function that truncates every word to the first 4 letters (see appendix for details)
def four_letters(string):
    words = string.split(' ')
    short_words=[word[0:4] for word in words]
    return(' '.join(short_words))

In [37]:
# Compare to passphrase above
short_passphrase = four_letters(passphrase)
print(short_passphrase)
algosdk.mnemonic.to_private_key(short_passphrase)   # same as above!

agai mont trib than alph bles dies drip movi awkw shoo deri acid lock brok titl vapo nigh salm fina enro trap poin abil pled


'Jdij0P2Ng16wJ0OHNASNt/OADZLj4otn5XxplSWd6zTeSrrq2ayN9slDQUBAP4On+k1RiJQNqEncS8G7LhTrog=='

#### A quick note on passphrases
+ Some software requires the entire words

## Appendix: how `four_letters()` works
A step-by-step construction of the function

In [38]:
string = 'Welcome to the WSC blochchain school'

In [39]:
string[0:4]                 # first four letters

'Welc'

In [40]:
string.split(' ')           # split at space --> create a list

['Welcome', 'to', 'the', 'WSC', 'blochchain', 'school']

In [41]:
# List expression
words = string.split(' ')
[word[0:4] for word in words]

['Welc', 'to', 'the', 'WSC', 'bloc', 'scho']

In [42]:
# Join the list to a string again
words = string.split(' ')
short_words=[word[0:4] for word in words]
print(' '.join(short_words))

Welc to the WSC bloc scho


In [43]:
# package as a function
def four_letters(string):
    words = string.split(' ')
    short_words=[word[0:4] for word in words]
    return(' '.join(short_words))

## Appendix: can you create your own Mnemonic?
* Idea: choose words that tell a story for easy memorizing
* Not possible: we would not be able to choose the correct 25th word
* Also not advisible: "hand picked" words are not sufficiently random and easier to crack